<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Tracking a Multi-step LLM Chain

In this exercise, you'll track a multi-step LLM chain with Opik. You can use OpenAI or open source models via LiteLLM.

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If OpenAI is called within one of these steps, the LLM call with be associated with that corresponding step:

# Imports & Configuration

In [ ]:
%pip install opik openai --quiet

In [ ]:
from opik import track
import opik
from opik.integrations.openai import track_openai
from openai import OpenAI
import getpass
import os

os.environ["OPIK_PROJECT_NAME"] = "Multi-step-Chain-Demo"

In [ ]:
# opik configs
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

In [ ]:
# openai configs
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
client = OpenAI()
openai_client = track_openai(client)

* Using open-source model from HuggingFace with LiteLLM instead of OpenAI above

In [1]:
!pip install opik litellm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.5 MB/s eta 0:00:00


In [2]:
import opik
import os
import getpass

# Define your OPIK project name to log traces with @track decorator
os.environ["OPIK_PROJECT_NAME"] = "Multi-step-Chain-Demo"

if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


Enter your Opik API key: ··········
Do you want to use "bluemusk" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


In [3]:
from litellm.integrations.opik.opik import OpikLogger
from opik.opik_context import get_current_span_data
from opik import track
import litellm

opik_logger = OpikLogger()
# In order to log LiteLLM traces to Opik, you will need to set the Opik callback
litellm.callbacks = [opik_logger]

In [4]:
# set Hugging Face API key to access meta-llama3
import os
import getpass

if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")

Enter your Hugging Face API key: ··········


# Define First Step

In [12]:
@track
def generate_meal(ingredient):
    prompt = f"Generate one example of a meal that can be made with {ingredient}."
    res = litellm.completion(
        model="huggingface/meta-llama/Llama-3.2-1B-Instruct",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Define Second Step

In [13]:
@track
def generate_recipe(meal):
    prompt = f"Generate a step-by-step recipe for {meal}"
    res = litellm.completion(
        model="huggingface/meta-llama/Llama-3.2-1B-Instruct",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Call Chain

In [14]:
@track
def generate_recipe_from_ingredient(ingredient):
    meal = generate_meal(ingredient)
    story = generate_recipe(meal)
    return story

generate_recipe_from_ingredient("garlic")

"Here's a step-by-step recipe for Garlic Chicken Fajitas:\n\n**Servings:** 4-6 people\n\n**Cooking Time:** 20-25 minutes\n\n**Prep Time:** 10 minutes\n\n**Total Time:** 30-35 minutes\n\n**Step-by-Step Instructions:**\n\n1. **Prepare the ingredients:**\n\t* Rinse the chicken breast and pat it dry with paper towels.\n\t* Peel and mince the garlic cloves.\n\t* Slice the onion"

# Try with your own example!

In [15]:
generate_recipe_from_ingredient(input("Enter an ingredient: "))

Enter an ingredient: Jollof rice


"Here's a step-by-step recipe for Jollof Rice with Chicken and Vegetables:\n\n**Servings:** 4-6 people\n\n**Cooking Time:** 30-40 minutes\n\n**Prep Time:** 15 minutes\n\n**Total Time:** 45-55 minutes\n\n**Step-by-Step Instructions:**\n\n**Step 1: Prepare the Chicken**\n\n1. In a large bowl, whisk together 2 tablespoons of vegetable oil, 1 teaspoon of salt, and 1"

## Trying another example not related to ingredient and meal

In [19]:
# First step

@track
def get_product(product):
    prompt = f"What is the best name to describe a company that makes {product}?"
    res = litellm.completion(
        model="huggingface/meta-llama/Llama-3.2-1B-Instruct",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

In [20]:
# Second step

@track
def get_company(company_name):
  prompt = f"Write a 20 words description for the following company: {company_name}"
  res = litellm.completion(
        model="huggingface/meta-llama/Llama-3.2-1B-Instruct",
        messages=[
            {"role": "user", "content": prompt}
        ]
  )
  return res.choices[0].message.content

In [21]:
# Call chain of both steps above

@track
def describe_company(product):
  company = get_product(product)
  description = get_company(company)
  return description

# Run the chain
describe_company('football')

"Here are 20-word descriptions for each of the company name suggestions:\n\n1. **Kickoff Sports**: Kickstart your passion for football with a company that embodies the excitement and energy of the game.\n2. **Field & Co.**: Experience the thrill of football with a company that's dedicated to providing top-notch gear and expert advice.\n3. **Gridiron Gear**: Elevate your game with a company that's all about delivering high-quality football equipment and expert guidance.\n4. **Kickoff"

* Note: refresh Opik UI to see updated projects